<a href="https://colab.research.google.com/github/yatinahujaahahaha/Unemployment-during-Recessions-A-Comparative-Analysis-of-the-Demographic-Disparities-across-Europe/blob/main/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:

!pip install pandas matplotlib seaborn


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



In [5]:
from google.colab import files

uploaded = files.upload()  # This will open a file picker

Saving estat_lfsq_urgacob.tsv to estat_lfsq_urgacob.tsv


In [6]:
# Step 1: Reading the TSV file
df = pd.read_csv("estat_lfsq_urgacob.tsv", sep="\t")

# Step 2: Spliting the combined column
# The first column has metadata: freq, unit, sex, age, c_birth, geo
df[['freq', 'c_birth', 'sex', 'age', 'unit', 'geo']] = df.iloc[:, 0].str.split(",", expand=True)

# Step 3: Renaming and reshaping
# Drop the original combined column (now redundant)
df = df.drop(columns=df.columns[0])

# Step 4: Melting the quarter columns into long format
df_long = df.melt(
    id_vars=['freq', 'unit', 'sex', 'age', 'c_birth', 'geo'],
    var_name='time_period',
    value_name='unemployment_raw'
)

# Step 5: Extracting numeric values
df_long['unemployment_rate'] = df_long['unemployment_raw'].str.extract(r'([0-9.]+)').astype(float)

# Step 6: Droping missing rates
df_long = df_long.dropna(subset=['unemployment_rate'])

# Step 7: Preview the cleaned data
df_long.head()



,freq,unit,sex,age,c_birth,geo,time_period,unemployment_raw,unemployment_rate
49,Q,PC,F,Y15-24,EU27_2020_FOR,EL,1998-Q1,52.1 bu,52.1
50,Q,PC,F,Y15-24,EU27_2020_FOR,ES,1998-Q1,63.1 u,63.1
87,Q,PC,F,Y15-39,EU27_2020_FOR,EL,1998-Q1,30.2 b,30.2
88,Q,PC,F,Y15-39,EU27_2020_FOR,ES,1998-Q1,35.0,35.0
91,Q,PC,F,Y15-39,EU27_2020_FOR,FR,1998-Q1,14.9,14.9


In [9]:
# 1. trailing spaces removed from all string columns
df_long = df_long.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

# 2. Extracting year and quarter
yr_q = df_long['time_period'].str.extract(r'(?P<year>\d{4})-?Q(?P<q>[1-4])').astype({'year': int, 'q': int})
df_long[['year','q']] = yr_q
df_long['quarter_str'] = df_long['year'].astype(str) + 'Q' + df_long['q'].astype(str)

# 3. Maping origin status
def map_origin(cb):
    u = str(cb).upper()
    if 'NAT' == u:  return 'native'
    if 'FOR' == u:  return 'foreign'
    return None
df_long['origin_status'] = df_long['c_birth'].apply(map_origin)

# 4. Filter
df_long = df_long[
    (df_long['age'] == 'Y15-64') &
    (df_long['sex'] == 'T') &
    (df_long['origin_status'].isin(['native','foreign']))
].copy()


df_long = df_long[df_long['quarter_str'] >= '2005Q1'].copy()

# Check results

print("\nOrigin status counts:", df_long['origin_status'].value_counts())
df_long



Origin status counts: origin_status
native     2821
foreign    2432
Name: count, dtype: int64


,freq,unit,sex,age,c_birth,geo,time_period,unemployment_raw,unemployment_rate,year,q,quarter_str,origin_status
569352,Q,PC,T,Y15-64,FOR,AT,2005-Q1,11.7 b,11.7,2005,1,2005Q1,foreign
569354,Q,PC,T,Y15-64,FOR,BE,2005-Q1,17.4 b,17.4,2005,1,2005Q1,foreign
569357,Q,PC,T,Y15-64,FOR,CY,2005-Q1,6.7 b,6.7,2005,1,2005Q1,foreign
569358,Q,PC,T,Y15-64,FOR,CZ,2005-Q1,14.6 b,14.6,2005,1,2005Q1,foreign
569359,Q,PC,T,Y15-64,FOR,DE,2005-Q1,18.0 b,18.0,2005,1,2005Q1,foreign
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182803,Q,PC,T,Y15-64,NAT,RO,2025-Q1,6.2,6.2,2025,1,2025Q1,native
2182804,Q,PC,T,Y15-64,NAT,RS,2025-Q1,9.6,9.6,2025,1,2025Q1,native
2182805,Q,PC,T,Y15-64,NAT,SE,2025-Q1,6.4,6.4,2025,1,2025Q1,native
2182806,Q,PC,T,Y15-64,NAT,SI,2025-Q1,3.8,3.8,2025,1,2025Q1,native


In [10]:
# Explicit core and periphery lists
core_countries = ["AT","BE","DK","FI","FR","DE","SE"]
periphery_countries = ["CY","EL","IT","PT","ES"]

# Map region group
def classify_region(geo):
    if geo in core_countries:
        return 'core'
    elif geo in periphery_countries:
        return 'periphery'
    else:
        return None  # for countries not in your study

df_long['region_group'] = df_long['geo'].apply(classify_region)

# Periphery dummy
df_long['periphery'] = (df_long['region_group'] == 'periphery').astype(int)

# Check mapping
print(df_long[['geo', 'region_group', 'periphery']].drop_duplicates().sort_values('geo'))
df_long



               geo region_group  periphery
569352          AT         core          0
1857353         BA         None          0
569354          BE         core          0
653275          BG         None          0
589481          CH         None          0
569357          CY    periphery          1
569358          CZ         None          0
569359          DE         core          0
569360          DK         core          0
569361        EA20         None          0
569362          EE         None          0
569363          EL    periphery          1
569364          ES    periphery          1
569365   EU27_2020         None          0
569366          FI         core          0
569367          FR         core          0
649868          HR         None          0
569369          HU         None          0
572790          IE         None          0
572791          IS         None          0
569372          IT    periphery          1
569373          LT         None          0
569374     

,freq,unit,sex,age,c_birth,geo,time_period,unemployment_raw,unemployment_rate,year,q,quarter_str,origin_status,region_group,periphery
569352,Q,PC,T,Y15-64,FOR,AT,2005-Q1,11.7 b,11.7,2005,1,2005Q1,foreign,core,0
569354,Q,PC,T,Y15-64,FOR,BE,2005-Q1,17.4 b,17.4,2005,1,2005Q1,foreign,core,0
569357,Q,PC,T,Y15-64,FOR,CY,2005-Q1,6.7 b,6.7,2005,1,2005Q1,foreign,periphery,1
569358,Q,PC,T,Y15-64,FOR,CZ,2005-Q1,14.6 b,14.6,2005,1,2005Q1,foreign,None,0
569359,Q,PC,T,Y15-64,FOR,DE,2005-Q1,18.0 b,18.0,2005,1,2005Q1,foreign,core,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182803,Q,PC,T,Y15-64,NAT,RO,2025-Q1,6.2,6.2,2025,1,2025Q1,native,None,0
2182804,Q,PC,T,Y15-64,NAT,RS,2025-Q1,9.6,9.6,2025,1,2025Q1,native,None,0
2182805,Q,PC,T,Y15-64,NAT,SE,2025-Q1,6.4,6.4,2025,1,2025Q1,native,core,0
2182806,Q,PC,T,Y15-64,NAT,SI,2025-Q1,3.8,3.8,2025,1,2025Q1,native,None,0


In [12]:
# Keeping only the countries in the thesis
keep_countries = core_countries + periphery_countries
df_long = df_long[df_long['geo'].isin(keep_countries)].copy()

df_long


,freq,unit,sex,age,c_birth,geo,time_period,unemployment_raw,unemployment_rate,year,q,quarter_str,origin_status,region_group,periphery
569352,Q,PC,T,Y15-64,FOR,AT,2005-Q1,11.7 b,11.7,2005,1,2005Q1,foreign,core,0
569354,Q,PC,T,Y15-64,FOR,BE,2005-Q1,17.4 b,17.4,2005,1,2005Q1,foreign,core,0
569357,Q,PC,T,Y15-64,FOR,CY,2005-Q1,6.7 b,6.7,2005,1,2005Q1,foreign,periphery,1
569359,Q,PC,T,Y15-64,FOR,DE,2005-Q1,18.0 b,18.0,2005,1,2005Q1,foreign,core,0
569360,Q,PC,T,Y15-64,FOR,DK,2005-Q1,11.4 b,11.4,2005,1,2005Q1,foreign,core,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182786,Q,PC,T,Y15-64,NAT,FI,2025-Q1,8.6,8.6,2025,1,2025Q1,native,core,0
2182787,Q,PC,T,Y15-64,NAT,FR,2025-Q1,7.0 d,7.0,2025,1,2025Q1,native,core,0
2182792,Q,PC,T,Y15-64,NAT,IT,2025-Q1,6.6,6.6,2025,1,2025Q1,native,periphery,1
2182802,Q,PC,T,Y15-64,NAT,PT,2025-Q1,6.3,6.3,2025,1,2025Q1,native,periphery,1


In [13]:

# Global Financial Crisis: 2008Q3 – 2009Q4
df_long['gfc'] = (
    ((df_long['year'] == 2008) & (df_long['q'] >= 3)) |
    (df_long['year'] == 2009)
).astype(int)

# Eurozone Debt Crisis: 2011Q3 – 2013Q2
df_long['eurozone'] = (
    ((df_long['year'] == 2011) & (df_long['q'] >= 3)) |
    (df_long['year'] == 2012) |
    ((df_long['year'] == 2013) & (df_long['q'] <= 2))
).astype(int)

# COVID-19 shock: 2020Q1 – 2021Q2
df_long['covid'] = (
    ((df_long['year'] == 2020) & (df_long['q'].between(1,4))) |
    ((df_long['year'] == 2021) & (df_long['q'] <= 2))
).astype(int)

# Pooled crisis dummy
df_long['shock_dummy'] = (
    (df_long['gfc'] == 1) |
    (df_long['eurozone'] == 1) |
    (df_long['covid'] == 1)
).astype(int)

# Check some rows
print(df_long[['periphery','geo','quarter_str','gfc','eurozone','covid','shock_dummy']].head(15))
df_long


        periphery geo quarter_str  gfc  eurozone  covid  shock_dummy
569352          0  AT      2005Q1    0         0      0            0
569354          0  BE      2005Q1    0         0      0            0
569357          1  CY      2005Q1    0         0      0            0
569359          0  DE      2005Q1    0         0      0            0
569360          0  DK      2005Q1    0         0      0            0
569363          1  EL      2005Q1    0         0      0            0
569364          1  ES      2005Q1    0         0      0            0
569366          0  FI      2005Q1    0         0      0            0
569367          0  FR      2005Q1    0         0      0            0
569372          1  IT      2005Q1    0         0      0            0
569382          1  PT      2005Q1    0         0      0            0
569385          0  SE      2005Q1    0         0      0            0
572772          0  AT      2005Q1    0         0      0            0
572774          0  BE      2005Q1 

,freq,unit,sex,age,c_birth,geo,time_period,unemployment_raw,unemployment_rate,year,q,quarter_str,origin_status,region_group,periphery,gfc,eurozone,covid,shock_dummy
569352,Q,PC,T,Y15-64,FOR,AT,2005-Q1,11.7 b,11.7,2005,1,2005Q1,foreign,core,0,0,0,0,0
569354,Q,PC,T,Y15-64,FOR,BE,2005-Q1,17.4 b,17.4,2005,1,2005Q1,foreign,core,0,0,0,0,0
569357,Q,PC,T,Y15-64,FOR,CY,2005-Q1,6.7 b,6.7,2005,1,2005Q1,foreign,periphery,1,0,0,0,0
569359,Q,PC,T,Y15-64,FOR,DE,2005-Q1,18.0 b,18.0,2005,1,2005Q1,foreign,core,0,0,0,0,0
569360,Q,PC,T,Y15-64,FOR,DK,2005-Q1,11.4 b,11.4,2005,1,2005Q1,foreign,core,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2182786,Q,PC,T,Y15-64,NAT,FI,2025-Q1,8.6,8.6,2025,1,2025Q1,native,core,0,0,0,0,0
2182787,Q,PC,T,Y15-64,NAT,FR,2025-Q1,7.0 d,7.0,2025,1,2025Q1,native,core,0,0,0,0,0
2182792,Q,PC,T,Y15-64,NAT,IT,2025-Q1,6.6,6.6,2025,1,2025Q1,native,periphery,1,0,0,0,0
2182802,Q,PC,T,Y15-64,NAT,PT,2025-Q1,6.3,6.3,2025,1,2025Q1,native,periphery,1,0,0,0,0


In [14]:


df_nf = df_long[
    (df_long['age'] == 'Y15-64') &
    (df_long['sex'] == 'T') &
    (df_long['c_birth'].isin(['NAT','FOR']))  # <-- crucial
].copy()

#sanity: check duplicates (should be False)
dups = (df_nf.groupby(['geo','quarter_str','origin_status']).size() > 1).any()
print("Duplicates per country-quarter-origin exist?", dups)

# Pivoting to wide without averaging subcategories
pivot = df_nf.pivot_table(
    index=['geo', 'quarter_str', 'region_group', 'periphery', 'shock_dummy'],
    columns='origin_status',
    values='unemployment_rate',
    aggfunc='first'   # each cell should be unique now
).reset_index()

# Droping rows missing either group
pivot = pivot.dropna(subset=['foreign', 'native'])

# Computing  gap & interaction
pivot['gap'] = pivot['foreign'] - pivot['native']
pivot['shock_periphery'] = pivot['shock_dummy'] * pivot['periphery']

pivot = pivot.rename(columns={'geo':'country'})

# check to confirm exact match
def check(country_code, q):
    r = pivot[(pivot['country'] == country_code) & (pivot['quarter_str'] == q)]
    if r.empty:
        print("No row for", country_code, q)
    else:
        r = r.iloc[0]
        print(f"{country_code} {q} -> foreign: {r['foreign']:.1f} native: {r['native']:.1f} gap: {r['gap']:.1f}")

check('AT', '2005Q1')  # should show 11.7 and 4.6 if present in my file
pivot

Duplicates per country-quarter-origin exist? False
AT 2005Q1 -> foreign: 11.7 native: 4.6 gap: 7.1


origin_status,country,quarter_str,region_group,periphery,shock_dummy,foreign,native,gap,shock_periphery
0,AT,2005Q1,core,0,0,11.7,4.6,7.1,0
1,AT,2005Q2,core,0,0,11.5,4.6,6.9,0
2,AT,2005Q3,core,0,0,10.8,4.7,6.1,0
3,AT,2005Q4,core,0,0,12.0,4.5,7.5,0
4,AT,2006Q1,core,0,0,12.8,4.7,8.1,0
...,...,...,...,...,...,...,...,...,...
963,SE,2024Q1,core,0,0,16.9,6.0,10.9,0
964,SE,2024Q2,core,0,0,17.6,6.3,11.3,0
965,SE,2024Q3,core,0,0,15.5,5.3,10.2,0
966,SE,2024Q4,core,0,0,15.5,5.2,10.3,0


In [26]:
import statsmodels.api as sm
import numpy as np


In [27]:
X = pivot[["shock_dummy", "periphery", "shock_periphery"]]
X = sm.add_constant(X)
y = pivot["gap"]


#  OLS
model = sm.OLS(y, X)

# Fit with clustered SEs by country
results_cluster = model.fit(
    cov_type='cluster',
    cov_kwds={'groups': pivot['country']}  # cluster variable
)

print(results_cluster.summary())

                            OLS Regression Results                            
Dep. Variable:                    gap   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     12.56
Date:                Fri, 05 Sep 2025   Prob (F-statistic):           0.000710
Time:                        16:00:03   Log-Likelihood:                -2425.7
No. Observations:                 968   AIC:                             4859.
Df Residuals:                     964   BIC:                             4879.
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               6.7237      0.718     

In [28]:
X

,const,shock_dummy,periphery,shock_periphery
0,1.0,0,0,0
1,1.0,0,0,0
2,1.0,0,0,0
3,1.0,0,0,0
4,1.0,0,0,0
...,...,...,...,...
963,1.0,0,0,0
964,1.0,0,0,0
965,1.0,0,0,0
966,1.0,0,0,0


In [29]:
import scipy.linalg

In [30]:
cfe = pd.get_dummies(pivot.loc[:,"country"]).astype("int")
tfe = pd.get_dummies(pivot.loc[:,"quarter_str"]).astype("int")
fe = pd.concat([cfe, tfe], axis=1)
pivot["shock_periphery"] = pivot["shock_dummy"] * pivot["periphery"]
matrix = pd.concat([pivot.loc[:, ["periphery", "shock_dummy", "shock_periphery"]], fe], axis=1)

from statsmodels.api import add_constant
matrix = add_constant(matrix)
rank = np.linalg.matrix_rank(matrix)
perf_mult = scipy.linalg.qr(matrix, pivoting=True)[2][rank:]
perf_mult_vars = matrix.iloc[:,perf_mult].columns
perf_mult_vars

Index(['2005Q3', 'DE', 'CY', '2020Q4'], dtype='object')

In [31]:
matrix

,const,periphery,shock_dummy,shock_periphery,AT,BE,CY,DE,DK,EL,...,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4,2025Q1
0,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
964,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
965,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
966,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [32]:
cols_to_drop = ['2005Q3', 'DE', 'CY', '2020Q4']
matrix_reduced = matrix.drop(columns=cols_to_drop)

In [33]:
matrix_reduced

,const,periphery,shock_dummy,shock_periphery,AT,BE,DK,EL,ES,FI,...,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4,2025Q1
0,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
964,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
965,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
966,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
y = pivot["gap"]

# X = independent variables (already reduced for collinearity)
X = matrix_reduced

# OLS
model = sm.OLS(y, X)

# Fit with clustered SEs by country
results_cluster = model.fit(
    cov_type='cluster',
    cov_kwds={'groups': pivot['country']}  # cluster variable
)

print(results_cluster.summary())

                            OLS Regression Results                            
Dep. Variable:                    gap   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.620
Method:                 Least Squares   F-statistic:                -2.062e+11
Date:                Fri, 05 Sep 2025   Prob (F-statistic):               1.00
Time:                        16:52:10   Log-Likelihood:                -2002.4
No. Observations:                 968   AIC:                             4191.
Df Residuals:                     875   BIC:                             4644.
Df Model:                          92                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               3.6786      0.871     

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 92, but rank is 11
  warnings.warn('covariance of constraints does not have full '
